In [66]:
import pandas as pd

import numpy as np

from sklearn.metrics import mean_squared_error

from skrub import TableVectorizer
import xgboost as xgb
from sklearn.pipeline import Pipeline


In [55]:
# Import the files
df_train = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/train.parquet")
df_test = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/final_test.parquet")
external_data = pd.read_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/external_data.csv")


In [56]:
external_data['date'].dtype

dtype('O')

In [46]:
external_data.head()

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,...,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,...,1500.0,2.0,3.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,...,480.0,4.0,6.0,2000.0,6.0,3.0,3000.0,NaN,NaN,NaN
3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,...,1740.0,3.0,3.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,...,330.0,4.0,6.0,570.0,7.0,6.0,810.0,NaN,NaN,NaN


In [47]:
print(external_data.shape)

(3322, 59)


In [48]:
unique_values = external_data.nunique()
print(unique_values)

numer_sta       1
date         3321
pmer          465
tend           99
cod_tend        9
dd             37
ff            109
t             345
td            281
u              77
vv           1230
ww             42
w1              9
w2              9
n              10
nbas           10
hbas           97
cl             11
cm              9
ch             10
pres          459
niv_bar         0
geop            0
tend24        353
tn12          217
tn24            0
tx12          279
tx24            0
tminsol         1
sw              0
tw              0
raf10         170
rafper        186
per             1
etat_sol        9
ht_neige        7
ssfrai          3
perssfrai       2
rr1            33
rr3            64
rr6            91
rr12          124
rr24          164
phenspe1        0
phenspe2        0
phenspe3        0
phenspe4        0
nnuage1         9
ctype1          9
hnuage1       279
nnuage2         8
ctype2          8
hnuage2       244
nnuage3         8
ctype3          8
hnuage3   

In [49]:
external_data['date'] = pd.to_datetime(external_data['date'])

first_three_days = external_data[external_data['date'].dt.date <= external_data['date'].dt.date.min() + pd.Timedelta(days=2)]

# Print the rows for the first three days
print("Rows for the first three days of the external data:")
first_three_days

Rows for the first three days of the external data:


,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
3082,7149,2020-09-01 00:00:00,102050,-10,8,340,1.6,285.75,282.55,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3083,7149,2020-09-01 03:00:00,101990,-60,6,290,1.1,283.95,282.05,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3084,7149,2020-09-01 06:00:00,102000,10,3,360,1.8,284.25,282.85,91,...,4000.0,1.0,0.0,7500.0,NaN,NaN,NaN,NaN,NaN,NaN
3085,7149,2020-09-01 09:00:00,101970,0,5,360,2.9,291.25,283.35,60,...,950.0,1.0,3.0,4500.0,NaN,NaN,NaN,NaN,NaN,NaN
3086,7149,2020-09-01 12:00:00,101850,-110,6,40,2.6,293.95,281.25,44,...,1410.0,5.0,NaN,2160.0,NaN,NaN,NaN,NaN,NaN,NaN
3087,7149,2020-09-01 15:00:00,101740,-110,6,40,4.0,293.65,279.95,41,...,1700.0,5.0,6.0,2300.0,NaN,NaN,NaN,NaN,NaN,NaN
3088,7149,2020-09-01 18:00:00,101760,10,3,20,3.0,292.15,280.55,47,...,1700.0,7.0,6.0,2400.0,NaN,NaN,NaN,NaN,NaN,NaN
3089,7149,2020-09-01 21:00:00,101880,110,3,10,2.7,289.35,279.75,53,...,1900.0,3.0,6.0,2400.0,NaN,NaN,NaN,NaN,NaN,NaN
3090,7149,2020-09-02 00:00:00,101920,30,2,360,3.2,287.75,280.75,63,...,2340.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3091,7149,2020-09-02 03:00:00,101910,-20,7,360,1.4,285.45,281.35,76,...,2280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
first_three_days_train = df_train[df_train['date'].dt.date <= df_train['date'].dt.date.min() + pd.Timedelta(days=2)]

# Print the rows for the first three days
print("Rows for the first three days of the external data:")
first_three_days_train.head(30)

Rows for the first three days of the external data:


,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585
48336,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2.0,2020-09-01 20:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.098612
48339,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 23:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48342,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-02 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48345,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,3.0,2020-09-02 11:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.386294
48348,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,5.0,2020-09-02 13:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.791759


What we can see : the days match between the external data set and the train and test sets BUT not the hour !! We will have to deal with this. 

In [51]:
print(df_train['date'].max())
print(df_train['date'].min())

2021-09-09 23:00:00
2020-09-01 01:00:00


In [52]:
print(df_test['date'].max())
print(df_test['date'].min())

2021-10-18 21:00:00
2021-09-10 01:00:00


In [53]:
print(external_data['date'].max())
print(external_data['date'].min())

2021-10-21 12:00:00
2020-09-01 00:00:00


In [58]:
# we convert 'date' column to datetime in all datasets
external_data['date'] = pd.to_datetime(external_data['date'])
df_train['date'] = pd.to_datetime(df_train['date']).astype('datetime64[ns]')
df_test['date'] = pd.to_datetime(df_test['date']).astype('datetime64[ns]')

# We select the relevant features from the external data that we want to add to our analsis :
weather_features = external_data[['date', 't', 'u', 'ff', 'pmer', 'tend', 'dd', 'ww']]

# Sort datasets by date
weather_features.sort_values('date', inplace=True)
df_train.sort_values('date', inplace=True)
df_test.sort_values('date', inplace=True)

# we merge the data together  by performing the nearest match in terms of date :
# training_set_merged = pd.merge(df_train, weather_features, on=['date'], how='left')
# testing_set_merged = pd.merge(df_test, weather_features, on=['date'], how='left')
# Perform nearest match
training_set_merged = pd.merge_asof(df_train, weather_features, on='date', direction='nearest')
testing_set_merged = pd.merge_asof(df_test, weather_features, on='date', direction='nearest')

/var/folders/kg/kfs42kh56l91cr2y6vt0_djr0000gn/T/ipykernel_10005/1530730449.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_features.sort_values('date', inplace=True)


In [59]:
print(weather_features['date'].dtype)
print(df_train['date'].dtype)
print(df_test['date'].dtype)

datetime64[ns]
datetime64[ns]
datetime64[ns]


In [60]:
training_set_merged. head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,t,u,ff,pmer,tend,dd,ww
0,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,1.0,2020-09-01 01:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,285.75,81,1.6,102050,-10,340,1
1,100047547-103047547,6 rue Julia Bartet SO-NE,100047547,6 rue Julia Bartet,2.0,2020-09-01 01:00:00,2018-11-28,"48.82636,2.30303",Y2H18086323,48.826360,2.303030,1.098612,285.75,81,1.6,102050,-10,340,1
2,100056047-SC,Pont de la Concorde S-N,100056047,Pont de la Concorde S-N,1.0,2020-09-01 01:00:00,2019-11-08,"48.86378,2.32003",Y2H19070356,48.863780,2.320030,0.693147,285.75,81,1.6,102050,-10,340,1
3,100047547-104047547,6 rue Julia Bartet NE-SO,100047547,6 rue Julia Bartet,4.0,2020-09-01 01:00:00,2018-11-28,"48.82636,2.30303",Y2H18086323,48.826360,2.303030,1.609438,285.75,81,1.6,102050,-10,340,1
4,100056223-SC,Pont des Invalides N-S,100056223,Pont des Invalides N-S,2.0,2020-09-01 01:00:00,2019-11-07,"48.86284,2.310345",Y2H19070365,48.862840,2.310345,1.098612,285.75,81,1.6,102050,-10,340,1


In [61]:
training_set_merged.isnull().sum()

counter_id                   0
counter_name                 0
site_id                      0
site_name                    0
bike_count                   0
date                         0
counter_installation_date    0
coordinates                  0
counter_technical_id         0
latitude                     0
longitude                    0
log_bike_count               0
t                            0
u                            0
ff                           0
pmer                         0
tend                         0
dd                           0
ww                           0
dtype: int64

dates match with the exception that the external_data max date is 21/10/2021 while test set ends on 18/10/2021. 3 useless days. 

In [62]:
# Extract the date feature on different time scales :

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # creation of a binary varible depicting if day in weekend
    X["is_weekend"] = np.where(X["weekday"] + 1 > 5, 1, 0)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

training_set_merged = _encode_dates(df_train)
testing_set_merged = _encode_dates(df_test)

In [63]:
X_train = training_set_merged.drop(columns=["bike_count", "log_bike_count"])
y_train = training_set_merged["log_bike_count"]

X_test = testing_set_merged.copy()

In [64]:
# Preprocessing pipeline
pipeline = Pipeline(
    steps=[
        ('preprocessor', TableVectorizer()),
        ('model', xgb.XGBRegressor()),
    ]
)

In [65]:
# Fit Pipeline to Training Data
pipeline.fit(X_train, y_train)

# Make Predictions on Test Data
y_predictions = pipeline.predict(X_test)


In [67]:
print(y_predictions)

[ 0.4174088   2.487812    2.3169124  ... -0.28238136  0.97111243
  2.1591904 ]


In [71]:
# We can print the RMSE on the training data :
y_train_predictions = pipeline.predict(X_train)
rmse_train = mean_squared_error(y_train, y_train_predictions, squared=False)
print(f"Training RMSE: {rmse_train}")

Training RMSE: 0.4027407573140838


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
pd.DataFrame(y_predictions, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/predictions_option_vsimple_with_weather.csv", index=False)